# Mangelfulle spørringer mot NVDB api V3? 

her sammenligner vi resultatet av spørringen `/vegobjekter/904?kommune=5001&veg(system)referanse=K` mot NVDB api V2 og V3.

In [1]:
import json
import pandas as pd
import os

In [2]:
mappe = 'stavanger_904'
filer = os.listdir(mappe)

In [3]:
manglergeom = []
fasit = []        

etterspurt_E = []
etterspurt_R = []
etterspurt_F = []
etterspurt_K = []
etterspurt_P = []
etterspurt_S = []


for eifil in filer: 
    if 'json' in eifil: 
        spurt_vegkat = eifil.split( '.')[0].split( '_')[-1]
        with open( os.path.join( mappe, eifil), encoding='utf-8') as f:
            data = json.load( f)
        for rad in data: 
            rad['spurt_vegkat'] = spurt_vegkat
        
        if 'mengdeuttak_v2' in eifil:
            fasit.extend( data)
        elif 'mangler_geometriv3' in eifil: 
            manglergeom.extend( data )            
            

manglergeom = pd.DataFrame( manglergeom )
fasit = pd.DataFrame( fasit )

# Har vi duplikater i de ugyldige V3-objektene? 

In [4]:
duplikat = manglergeom[ manglergeom.duplicated(subset='id', keep=False) ]

In [5]:
print( len( duplikat), 'duplikater av', len( manglergeom), 'ugyldige og', len(fasit), 'totalt')

4 duplikater av 155 ugyldige og 5648 totalt


Relativt uinteressant, iallfall inntil  videre

# Lær mer om ugyldige data

### Stemmer ønsket vegkategori med det vi fant? 

In [6]:
mangler = pd.merge( left=manglergeom, right=fasit, left_on='id', right_on='vegobjektid', how='inner')

In [8]:
mangler.columns

Index(['id', 'href_x', 'metadata', 'egenskaper', 'lokasjon', 'vegsegmenter',
       'relasjoner', 'spurt_vegkat_x', 'apiversjon', 'vegobjektid', 'href_y',
       'versjon', 'startdato', 'geometri', 'geometritype', 'wkt_punkt',
       'kommuner', 'vegkat', 'vref', 'stedfesting_antall',
       'stedfesting_utbredelse', 'stedfesting_unikeid', 'spurt_vegkat_y'],
      dtype='object')

In [11]:
mangler[ mangler['spurt_vegkat_x'] == mangler['vegkat']]

,id,href_x,metadata,egenskaper,lokasjon,vegsegmenter,relasjoner,spurt_vegkat_x,apiversjon,vegobjektid,...,geometri,geometritype,wkt_punkt,kommuner,vegkat,vref,stedfesting_antall,stedfesting_utbredelse,stedfesting_unikeid,spurt_vegkat_y


### Er det en vegkategori som peker seg ut?

In [7]:
mangler.vegkat.value_counts()

P    107
K     41
F      4
E      2
S      1
Name: vegkat, dtype: int64

Sjekker ut den skogsbilvegen og de to europaveg-objektene 